In [8]:
from jinja2 import Template
import random

# filepath = "workflows/txt2img_with_hires_fix.txt"
# filepath = "workflows/test1.json"

filepath = "/Users/georgebennett/Documents/Code/script_writer/workflows/workflow_api.json"
with open(filepath, 'r') as file:
    content = file.read()
    # print(content)

template = Template(content)

# Create a dictionary with values for the template variables
random_number = random.randint(1000, 1000000)
data_dict = {
    'prompt': "a man in",
    "batch_size": 1,
    'seed': random_number
    }

# Render the template with the values from the dictionary
rendered_template = template.render(data_dict)


print(type(rendered_template))

<class 'str'>


In [9]:
import json
from jinja2 import Template
import random


def render_from_template (template_content, data_dict):
    template = Template(template_content)
    rendered_template = template.render(data_dict)
    rendered_template = json.loads(rendered_template)
    return rendered_template



random_number = random.randint(1000, 1000000)
data_dict = {
    'prompt': "a man in",
    "batch_size": 1,
    'seed': random_number
    }

filepath = "/Users/georgebennett/Documents/Code/script_writer/workflows/workflow_api.json"
with open(filepath, 'r') as file:
    template_content = file.read()

rendered_template = render_from_template(template_content, data_dict)
print(rendered_template)





{'3': {'inputs': {'seed': 550771, 'steps': 12, 'cfg': 8, 'sampler_name': 'dpmpp_sde', 'scheduler': 'normal', 'denoise': 1, 'model': ['16', 0], 'positive': ['6', 0], 'negative': ['7', 0], 'latent_image': ['5', 0]}, 'class_type': 'KSampler'}, '5': {'inputs': {'width': 768, 'height': 768, 'batch_size': 1}, 'class_type': 'EmptyLatentImage'}, '6': {'inputs': {'text': 'Close up documentary image of a sexy a young blonde 20 y.o. italian 1girl, in a crowded office, nude, sat on an office chair legs spread wide. knees up. smiling', 'clip': ['16', 1]}, 'class_type': 'CLIPTextEncode'}, '7': {'inputs': {'text': 'bad hands, text, watermark\n', 'clip': ['16', 1]}, 'class_type': 'CLIPTextEncode'}, '8': {'inputs': {'samples': ['3', 0], 'vae': ['16', 2]}, 'class_type': 'VAEDecode'}, '9': {'inputs': {'filename_prefix': 'ComfyUI', 'images': ['8', 0]}, 'class_type': 'SaveImage'}, '10': {'inputs': {'upscale_method': 'nearest-exact', 'width': 1152, 'height': 1152, 'crop': 'disabled', 'samples': ['3', 0]}, '

In [15]:
import json
from urllib import request, parse
import random
from urllib import request, error
import uuid



# def queue_prompt(prompt):
#     p = {"prompt": prompt, "client_id": client_id}
#     data = json.dumps(p).encode('utf-8')
#     req =  urllib.request.Request("http://{}/prompt".format(server_address), data=data)
#     return json.loads(urllib.request.urlopen(req).read())


def queue_prompt_simple(prompt):
    client_id = str(uuid.uuid4())
    # p = {"prompt": prompt}
    p = {"prompt": prompt, "client_id": client_id}
    data = json.dumps(p).encode('utf-8')
    print(data)
    try:
        req =  request.Request("https://b27b-81-148-229-203.ngrok-free.app/prompt", data=data)
        request.urlopen(req)

    except error.HTTPError as e:
        print(f"HTTP Error: {e.code}")
        print(f"Message: {e.reason}")
        print(f"URL: {e.url}")

        
prompt = json.loads(rendered_template)
# print(type(prompt))
# queue_prompt_simple(prompt)



TypeError: the JSON object must be str, bytes or bytearray, not dict

In [14]:
import uuid
import json
import urllib.request
import urllib.parse
import time
import requests


server_address = "b27b-81-148-229-203.ngrok-free.app"
client_id = str(uuid.uuid4())
print(client_id)


def queue_prompt(prompt):
    p = {"prompt": prompt, "client_id": client_id}
    data = json.dumps(p)
    req_url = f"https://{server_address}/prompt"
    response = requests.post(req_url, data=data)
    return response.json()



# Function to get an image from the server
def get_image(filename, subfolder, folder_type):
    params = {"filename": filename, "subfolder": subfolder, "type": folder_type}
    response = requests.get(f"https://{server_address}/view", params=params)
    return response.content

# Function to get the history of a prompt
def get_history(prompt_id):
    response = requests.get(f"https://{server_address}/history/{prompt_id}")
    return response.json()


def check_status(prompt_id):
    response = requests.get(f'https://{server_address}/prompt')
    if response.status_code == 200:
        data = json.loads(response.text)
        print(data)
        queue_remaining=data['exec_info']['queue_remaining']
        if queue_remaining == 0:
            print("done")
            return {'status': 'done'}
        else:
            print("not done")
            return {'status': 'not done'}



def get_images(prompt):
    # 1. Queue the prompt and get a prompt ID.
    queue_response = queue_prompt(prompt)
    prompt_id = queue_response['prompt_id']
    print(f"prompt_id: {prompt_id}")
    
    # Initialize an empty dictionary to store output images.
    output_images = {}

    # 2. Poll the server until the job corresponding to the prompt ID is done.
    while True:
        # Check the status of the prompt job.
        status = check_status(prompt_id)
        
        # If the status is 'done', break out of the loop.
        if status['status'] == 'done':
            break

        # Wait for 3 seconds before polling again.
        time.sleep(2)

    # 3. Retrieve the output history corresponding to the prompt ID.
    history = get_history(prompt_id)[prompt_id]
    # print("HISTORY")
    # print(history)

    # 4. Loop through the output history to gather image data.
    for node_id, node_output in history['outputs'].items():
        # Initialize an empty list to store images for each node.
        images_output = []
        
        # Check if 'images' key exists in the output for the current node.
        if 'images' in node_output:
            # Loop through each image and get its data.
            for image in node_output['images']:
                filename = image['filename']
                subfolder = image['subfolder']
                image_type = image['type']
                
                # Fetch the image data.
                image_data = get_image(filename, subfolder, image_type)
                
                # Append the image data to the list.
                images_output.append(image_data)
        
        # Store the list of images for the current node in the output_images dictionary.
        output_images[node_id] = images_output

    # 5. Return the output_images dictionary.
    return output_images


# Main function to get images based on a prompt

# prompt = json.loads(rendered_template)
# print(prompt)
images = get_images(prompt)



# # Code to display the output images:
# for node_id in images:
#     for image_data in images[node_id]:
#         from PIL import Image
#         import io
#         image = Image.open(io.BytesIO(image_data))
#         image.show()


83602fec-3ae5-4a25-b5d0-1bdf6b0f0d41


NameError: name 'prompt' is not defined

In [137]:
import json

def modify_json(json_obj, target_class, target_key, new_value, nth_instance=1, counter=0):
    """
    Modifies a JSON object in-place based on the specified target_class, target_key, and new_value.

    Args:
    - json_obj (dict): The JSON object to modify.
    - target_class (str): The target class_type to look for.
    - target_key (str): The key within the target class_type to change.
    - new_value: The new value to assign to target_key.
    - nth_instance (int): The nth instance of target_class to modify.
    - counter (int): A counter variable to keep track of the number of instances found.

    Returns:
    - counter (int): Updated counter variable.
    """
    for key, value in json_obj.items():
        if isinstance(value, dict):
            if value.get('class_type') == target_class:
                counter += 1
                if counter == nth_instance:
                    if target_key in value['inputs']:
                        value['inputs'][target_key] = new_value

            counter = modify_json(value, target_class, target_key, new_value, nth_instance, counter)

    return counter

   
   
# Target class_type, key, and new value
target_class = "SaveImage"
target_key = "filename_prefix"
new_value = "yolo"
nth_instance = 1

# Modify the JSON data
modify_json(prompt, target_class, target_key, new_value, nth_instance)

# Print modified JSON data
print(json.dumps(prompt, indent=4))


{
    "3": {
        "inputs": {
            "seed": 938378900260061,
            "steps": 20,
            "cfg": 8,
            "sampler_name": "euler",
            "scheduler": "normal",
            "denoise": 1,
            "model": [
                "4",
                0
            ],
            "positive": [
                "6",
                0
            ],
            "negative": [
                "7",
                0
            ],
            "latent_image": [
                "5",
                0
            ]
        },
        "class_type": "KSampler"
    },
    "4": {
        "inputs": {
            "ckpt_name": "protovisionXLHighFidelity3D_beta0520Bakedvae.safetensors"
        },
        "class_type": "CheckpointLoaderSimple"
    },
    "5": {
        "inputs": {
            "width": 512,
            "height": 512,
            "batch_size": 1
        },
        "class_type": "EmptyLatentImage"
    },
    "6": {
        "inputs": {
            "text": "a sexy man"